# 1. Configurar ambiente

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import spacy
!python -m spacy download es_core_news_md
nlp = spacy.load('es_core_news_md')
!pip install jellyfish
import jellyfish
!pip install transformers
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
from bs4 import BeautifulSoup
import requests
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#2. Tratamiento de datos

In [ ]:
#Función para encontrar la raiz de las palabras
def raiz(palabra):
  radio=0
  palabra_encontrada=palabra
  for word in lista_verbos:
    confianza = jellyfish.jaro_winkler(palabra, word)
    if (confianza>=0.93 and confianza>=radio):
      radio=confianza
      palabra_encontrada=word
  return palabra_encontrada

#Función para tratar los textos
def tratamiento_texto(texto):
  trans = str.maketrans('áéíóú','aeiou')
  texto = texto.lower()
  texto = texto.translate(trans)
  texto = " ".join(texto.split())
  return texto

#Función para reemplazar el final de una palabra por 'r'
def reemplazar_terminacion(palabra):
  patron = r"(es|me|as|te|ste)$"
  nueva_palabra = re.sub(patron, "r", palabra)
  return nueva_palabra.split()[0]

#Función para adicionar o eliminar tokens
def revisar_tokens(texto, tokens):
  if len(tokens)==0:
    if [x for x in ['elprofealejo', 'el profe alejo', 'profe alejo', 'profealejo'] if x in tratamiento_texto(texto)]: tokens.append('elprofealejo')
    elif [x for x in ['cientifico de datos', 'data scientist'] if x in tratamiento_texto(texto)]: tokens.append('datascientist')
    elif [x for x in ['ciencia de datos', 'data science'] if x in tratamiento_texto(texto)]: tokens.append('datascience')
    elif [x for x in ['big data', 'bigdata'] if x in tratamiento_texto(texto)]: tokens.append('bigdata')
  else:
    elementos_a_eliminar = ["profe", "alejo", "profealejo", "cual", "que", "quien", "cuanto", "cuando", "como"]
    if 'hablame' in texto and 'hablar' in tokens: tokens.remove('hablar')
    elif 'cuentame' in texto and 'contar' in tokens: tokens.remove('contar') 
    elif 'hago' in texto and 'hacer' in tokens: tokens.remove('hacer') 
    elif 'entiendes' in texto and 'entender' in tokens: tokens.remove('entender') 
    elif 'sabes' in texto and 'saber' in tokens: tokens.remove('saber')
    tokens = [x.replace('datar','data').replace('datos','dato') for x in tokens if x not in elementos_a_eliminar]
  return tokens

#Función para devolver los tokens normalizados del texto
def normalizar(texto):
  tokens=[]
  tokens=revisar_tokens(texto, tokens)
  if 'elprofealejo' in tokens:
    texto = ' '.join(texto.split()[:15])
  else:
    texto = ' '.join(texto.split()[:25])

  doc = nlp(texto)
  for t in doc:
    lemma=diccionario_irregulares.get(t.text, t.lemma_.split()[0])
    lemma=re.sub(r'[^\w\s+\-*/]', '', lemma)
    if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
      if t.pos_=='VERB':
        lemma = reemplazar_terminacion(lemma)
        tokens.append(raiz(tratamiento_texto(lemma)))
      else:
        tokens.append(tratamiento_texto(lemma))

  tokens = list(dict.fromkeys(tokens))
  tokens = list(filter(None, tokens))
  tokens = revisar_tokens(texto, tokens)
  tokens = ' '.join(tokens)
  return tokens

# 3. Cargar bases de conocimiento

In [ ]:
#Importando verbos en español
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
trans = str.maketrans('áéíóú','aeiou')
lista_verbos=[]
url = ['https://www.ejemplos.co/verbos-mas-usados-en-espanol/',
       'https://www.ejemplos.co/verbos-predicativos/',
       'https://www.ejemplos.co/verbos-personales/',
       'https://www.ejemplos.co/verbos-irregulares/',
       'https://www.ejemplos.co/verbos/',
       'https://www.ejemplos.co/100-ejemplos-de-verbos-regulares/',
       'https://www.ejemplos.co/verbos-del-decir/',
       'https://www.ejemplos.co/verbos-con-a/',
       'https://www.ejemplos.co/verbos-con-b/',
       'https://www.ejemplos.co/verbos-con-c/',
       'https://www.ejemplos.co/verbos-con-d/',
       'https://www.ejemplos.co/verbos-con-e/',
       'https://www.ejemplos.co/verbos-con-f/',
       'https://www.ejemplos.co/verbos-con-g/',
       'https://www.ejemplos.co/verbos-con-h/',
       'https://www.ejemplos.co/verbos-con-i/',
       'https://www.ejemplos.co/verbos-con-j/',
       'https://www.ejemplos.co/verbos-con-k/',
       'https://www.ejemplos.co/verbos-con-l/',
       'https://www.ejemplos.co/verbos-con-ll/',
       'https://www.ejemplos.co/verbos-con-m/',
       'https://www.ejemplos.co/verbos-con-n/',
       'https://www.ejemplos.co/verbos-con-o/',
       'https://www.ejemplos.co/verbos-con-p/',
       'https://www.ejemplos.co/verbos-con-q/',
       'https://www.ejemplos.co/verbos-con-r/',
       'https://www.ejemplos.co/verbos-con-s/',
       'https://www.ejemplos.co/verbos-con-t/',
       'https://www.ejemplos.co/verbos-con-u/',
       'https://www.ejemplos.co/verbos-con-v/',
       'https://www.ejemplos.co/verbos-con-w/',
       'https://www.ejemplos.co/verbos-con-x/',
       'https://www.ejemplos.co/verbos-con-y/',
       'https://www.ejemplos.co/verbos-con-z/']

for i in range(len(url)):
  try:
    respuesta = requests.get(url[i], headers=headers)
    respuesta = respuesta.content.decode('utf-8')
    bases = pd.read_html(respuesta)
    for i, df in enumerate(bases):
      for idx,row in bases[i].iterrows():     
          _ = [lista_verbos.append(re.sub(r"\((.*?)\)", '', x.lower()).strip().translate(trans)) for x in row[0].split('/')]
          _ = [lista_verbos.append(re.sub(r"\((.*?)\)", '', x.lower()).strip().translate(trans)) for x in row[1].split('/')]
          _ = [lista_verbos.append(re.sub(r"\((.*?)\)", '', x.lower()).strip().translate(trans)) for x in row[2].split('/')]
  except Exception:
    continue
lista_verbos = [elemento for elemento in lista_verbos if (elemento=='ir' or len(elemento)!=2)]
nuevos_verbos = ['costar', 'referir', 'datar']
lista_verbos.extend(nuevos_verbos)
lista_verbos=list(set(lista_verbos))

# Definir una lista de verbos irregulares y sus conjugaciones en pasado, presente, futuro, imperfecto, pretérito y condicional
verbos_irregulares = [
    ('ser', 'soy', 'eres', 'seras', 'eras', 'es', 'serias'),
    ('estar', 'estuviste', 'estas', 'estaras', 'estabas', 'estuviste', 'estarias'),
    ('ir', 'fuiste', 'vas', 'iras', 'ibas', 'fuiste', 'irias'),
    ('ir', 'fuiste', 'vaya', 'iras', 'ibas', 'fuiste', 'irias'),
    ('tener', 'tuviste', 'tienes', 'tendras', 'tenias', 'tuviste', 'tendrias'),
    ('hacer', 'hiciste', 'haces', 'haras', 'hacias', 'hiciste', 'harias'),
    ('decir', 'dijiste', 'dices', 'diras', 'decias', 'dijiste', 'dirias'),
    ('decir', 'dimar', 'dime', 'digame', 'dimir', 'dimo', 'dimiria'),
    ('poder', 'pudiste', 'puedes', 'podras', 'podias', 'pudiste', 'podrias'),
    ('saber', 'supiste', 'sabes', 'sabras', 'sabias', 'supiste', 'sabrias'),
    ('poner', 'pusiste', 'pones', 'pondras', 'ponias', 'pusiste', 'pondrias'),
    ('ver', 'viste', 'ves', 'veras', 'veias', 'viste', 'verias'),
    ('dar', 'diste', 'das', 'daras', 'dabas', 'diste', 'darias'),
    ('dar', 'damar', 'dame', 'daras', 'dabas', 'darme', 'darias'),
    ('venir', 'viniste', 'vienes', 'vendras', 'venias', 'viniste', 'vendrias'),
    ('haber', 'haya', 'has', 'habras', 'habias', 'hubiste', 'habrias'),
    ('caber', 'cupiste', 'cabes', 'cabras', 'cabias', 'cupiste', 'cabrias'),
    ('valer', 'valiste', 'vales', 'valdras', 'valias', 'valiste', 'valdrias'),
    ('querer', 'quisiste', 'quieres', 'querras', 'querias', 'quisiste', 'querrias'),
    ('llegar', 'llegaste', 'llegares', 'llegaras', 'llegarias', 'llegaste', 'llegarrias'),
    ('hacer', 'hiciste', 'haces', 'haras', 'hacias', 'hiciste', 'harias'),
    ('decir', 'dijiste', 'dices', 'diras', 'decias', 'dijiste', 'dirias'),
    ('poder', 'pudiste', 'puedes', 'podras', 'podias', 'pudiste', 'podria'),
    ('contar', 'contaste', 'cuentas', 'contaras', 'contabas', 'cuentame', 'contarias'),
    ('saber', 'supiste', 'sabes', 'sabras', 'sabias', 'supiste', 'sabrias'),
    ('costar', 'cuesta', 'cuestan', 'costo', 'costaria', 'costarian', 'cuestas'),
    ('durar', 'duraste', 'duro', 'duraras', 'durabas', 'duraste', 'durarias')
]

# Crear el DataFrame
diccionario_irregulares = {}
df = pd.DataFrame(verbos_irregulares, columns=['Verbo', 'Pasado', 'Presente', 'Futuro', 'Imperfecto', 'Pretérito', 'Condicional'])
for columna in df.columns:
  if columna != 'Verbo':
    for valor in df[columna]:
      diccionario_irregulares[valor] = df.loc[df[columna] == valor, 'Verbo'].values[0]

#Importando bases de dialogo fluído
txt_folder_path = '/content/dialogos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
lista_dialogos, lista_dialogos_respuesta, lista_tipo_dialogo = [],[],[]
for idx in range(len(lista_documentos)):
  f=open(txt_folder_path+'/'+lista_documentos[idx], 'r', encoding='utf-8', errors='ignore')
  flag,posicion = True,0
  for line in f.read().split('\n'):
    if flag:
      line = tratamiento_texto(line)
      line = re.sub(r'[^\w\s+\-*/]', '', line)
      lista_dialogos.append(line)
      lista_tipo_dialogo.append(lista_documentos[idx].replace('.txt', ''))
    else:
      lista_dialogos_respuesta.append(line)
      posicion+=1
    flag=not flag

#Creando Dataframe de diálogos
datos = {'dialogo':lista_dialogos,'respuesta':lista_dialogos_respuesta,'tipo':lista_tipo_dialogo,'interseccion':0,'similarity':0,'jaro_winkler':0,'probabilidad':0}
df_dialogo = pd.DataFrame(datos)
df_dialogo = df_dialogo.drop_duplicates(keep='first')
df_dialogo.reset_index(drop=True, inplace=True)

# 4. Normalizando dialogos

In [ ]:
#Normalizando las frases
label_encoder = LabelEncoder()
df_dialogo['palabras'] = df_dialogo['dialogo'].apply(normalizar)
df_dialogo['tipo_num'] = label_encoder.fit_transform(df_dialogo['tipo'])
df_dialogo = df_dialogo[df_dialogo.palabras.values!='']
df_dialogo

,dialogo,respuesta,tipo,interseccion,similarity,jaro_winkler,probabilidad,palabras,tipo_num
0,estoy bien,¡Es bueno saber esto!,Sentimiento,0,0,0,0,estar bien,14
1,estoy bien y tu,Estoy bien también. ¡Es bueno saberlo!,Sentimiento,0,0,0,0,estar bien,14
2,muy bieny vos,Yo también estoy muy bien. ¡Gracias por pregun...,Sentimiento,0,0,0,0,mucho bieny vo,14
3,puedes dar consejos sentimentales,"No, los sentimientos humanos son muy complicados.",Sentimiento,0,0,0,0,poder dar consejo sentimental,14
4,puedes mentir,"No, es malo mentir.",Sentimiento,0,0,0,0,poder mentir,14
...,...,...,...,...,...,...,...,...,...
1146,no me gusto tu respuesta,Mis disculpas. Ya actualicé mi sistema. ¿Podrí...,Error,0,0,0,0,no yo gustar respuesta,7
1147,eso no era mi pregunta,"Lo siento, ¿cuál era tu pregunta?",Error,0,0,0,0,ese no ser pregunta,7
1148,yo no pregunte eso,"Lo siento, ¿cuál era tu pregunta?",Error,0,0,0,0,yo no preguntar ese,7
1149,no es la respuesta,Lamento si mi respuesta no fue relevante. Ya a...,Error,0,0,0,0,no ser respuesta,7


In [ ]:
# Imprimir diccionario
relacion_diccionario = {}

# Iterar sobre las filas del DataFrame
for tipo, tipo_num in zip(df_dialogo['tipo'], df_dialogo['tipo_num']):
    relacion_diccionario[tipo_num] = tipo

# Imprimir el diccionario
print(relacion_diccionario)

{14: 'Sentimiento', 13: 'Saludos', 10: 'Nombre', 9: 'Identidad', 6: 'ElProfeAlejo', 1: 'Aprendizaje', 8: 'Funcion', 15: 'Usuario', 11: 'Origen', 5: 'Edad', 0: 'Agradecimiento', 3: 'Continuacion', 2: 'Contacto', 4: 'Despedida', 12: 'Otros', 7: 'Error'}


# 5. Entrenando con Naive Bayes

In [ ]:
# Separar los datos en características (X) y etiquetas (y)
X = df_dialogo['palabras']
y = df_dialogo['tipo_num']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorizar los datos de texto
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Entrenar el clasificador de Naive Bayes
modelo_NB = MultinomialNB()
modelo_NB.fit(X_train_vect, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = modelo_NB.predict(X_test_vect)

# Calcular el accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Precisión:", accuracy)

Precisión: 0.7


In [ ]:
# Calcular la precisión por clase
unique_classes = df_dialogo['tipo_num'].unique()
for cls in unique_classes:
    cls_indices = y_test == cls
    cls_accuracy = accuracy_score(y_test[cls_indices], y_pred[cls_indices])
    print("Accuracy para la clase", df_dialogo[df_dialogo.tipo_num == cls]['tipo'].unique()[0], ":", cls_accuracy)

Accuracy para la clase Continuacion : 0.5
Accuracy para la clase Nombre : 0.16666666666666666
Accuracy para la clase Contacto : 0.125
Accuracy para la clase Saludos : 0.9117647058823529
Accuracy para la clase Sentimiento : 0.45454545454545453
Accuracy para la clase Identidad : 0.6153846153846154
Accuracy para la clase Usuario : 0.7
Accuracy para la clase ElProfeAlejo : 0.5
Accuracy para la clase Aprendizaje : 0.2
Accuracy para la clase Agradecimiento : 0.7
Accuracy para la clase Edad : 0.5
Accuracy para la clase Despedida : 0.5
Accuracy para la clase Origen : 0.5
Accuracy para la clase Otros : 0.9565217391304348
Accuracy para la clase Error : 0.5
Accuracy para la clase Funcion : 0.5333333333333333


In [ ]:
# Procesando la nueva frase
frase = normalizar('como haces para aprender tan rapido?')
nueva_frase_vect = vectorizer.transform([frase])

# Realizar la predicción
prediccion = modelo_NB.predict(nueva_frase_vect)

print("La frase", frase, "se clasifica como: ", df_dialogo[df_dialogo.tipo_num == prediccion[0]]['tipo'].unique()[0])

La frase como hacer aprender tanto rapido se clasifica como:  Origen


# 6. Entrenando con Random Forest

In [ ]:
# Separar los datos en características (X) y etiquetas (y)
X = df_dialogo['palabras']
y = df_dialogo['tipo_num']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorizar los datos de texto
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Entrenar el clasificador Random Forest
Modelo_RF = RandomForestClassifier()
Modelo_RF.fit(X_train_vect, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = Modelo_RF.predict(X_test_vect)

# Calcular el accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Precisión:", accuracy)

Precisión: 0.782608695652174


In [ ]:
# Calcular la precisión por clase
unique_classes = df_dialogo['tipo_num'].unique()
for cls in unique_classes:
    cls_indices = y_test == cls
    cls_accuracy = accuracy_score(y_test[cls_indices], y_pred[cls_indices])
    print("Accuracy para la clase", df_dialogo[df_dialogo.tipo_num == cls]['tipo'].unique()[0], ":", cls_accuracy)

Accuracy para la clase Continuacion : 0.5
Accuracy para la clase Nombre : 0.8333333333333334
Accuracy para la clase Contacto : 0.5
Accuracy para la clase Saludos : 0.9117647058823529
Accuracy para la clase Sentimiento : 0.6818181818181818
Accuracy para la clase Identidad : 0.9230769230769231
Accuracy para la clase Usuario : 0.9
Accuracy para la clase ElProfeAlejo : 0.75
Accuracy para la clase Aprendizaje : 0.2
Accuracy para la clase Agradecimiento : 0.75
Accuracy para la clase Edad : 1.0
Accuracy para la clase Despedida : 0.5
Accuracy para la clase Origen : 0.625
Accuracy para la clase Otros : 0.855072463768116
Accuracy para la clase Error : 1.0
Accuracy para la clase Funcion : 0.7333333333333333


In [ ]:
# Procesando la nueva frase
frase = normalizar('como haces para aprender tan rapido?')
nueva_frase_vect = vectorizer.transform([frase])

# Realizar la predicción
prediccion = Modelo_RF.predict(nueva_frase_vect)

print("La frase", frase, "se clasifica como: ", df_dialogo[df_dialogo.tipo_num == prediccion[0]]['tipo'].unique()[0])

La frase como hacer aprender tanto rapido se clasifica como:  Otros


# 7. Entrenando con Transformers

In [ ]:
# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
df_train, df_test = train_test_split(df_dialogo, test_size=0.2, random_state=42)

# Cargar el modelo preentrenado de BERT para clasificación en español
model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df_dialogo['tipo_num'].nunique())
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokenizar y codificar las frases de entrenamiento
train_inputs = tokenizer.batch_encode_plus(
    df_train['palabras'].tolist(),
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Tokenizar y codificar las frases de prueba
test_inputs = tokenizer.batch_encode_plus(
    df_test['palabras'].tolist(),
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Preparar los datos de entrenamiento y prueba
train_data = torch.utils.data.TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], torch.tensor(df_train['tipo_num'].tolist()))
test_data = torch.utils.data.TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], torch.tensor(df_test['tipo_num'].tolist()))

# Definir el optimizador y la función de pérdida
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Entrenamiento del modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)

for epoch in range(5):  # Número de épocas de entrenamiento
    total_loss = 0

    for batch in train_dataloader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print("Epoch:", epoch + 1, "Loss:", total_loss)

# Evaluación del modelo
model.eval()
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

with torch.no_grad():
    predictions = []
    true_labels = []

    for batch in test_dataloader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        outputs = model(input_ids, attention_mask=attention_mask)

        _, predicted_labels = torch.max(outputs.logits, dim=1)

        predictions.extend(predicted_labels.tolist())
        true_labels.extend(labels.tolist())

accuracy = accuracy_score(true_labels, predictions)
print("Precisión:", accuracy)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuc

Epoch: 1 Loss: 134.830393075943
Epoch: 2 Loss: 107.36084604263306
Epoch: 3 Loss: 80.1585385799408
Epoch: 4 Loss: 58.961489140987396
Epoch: 5 Loss: 42.943987771868706
Precisión: 0.7467248908296943


In [ ]:
# Guardar el modelo entrenado
ruta_modelo = '/content/modelo'
model.save_pretrained(ruta_modelo)
tokenizer.save_pretrained(ruta_modelo)

('/content/modelo2/tokenizer_config.json',
 '/content/modelo2/special_tokens_map.json',
 '/content/modelo2/vocab.txt',
 '/content/modelo2/added_tokens.json')

In [ ]:
#Cargar el modelo entrenado
ruta_modelo = '/content/modelo'
Modelo_TF = BertForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer_TF = BertTokenizer.from_pretrained(ruta_modelo)

In [ ]:
# Calcular la precisión por clase
unique_classes = df_dialogo['tipo_num'].unique()

for class_label in unique_classes:
    # Filtrar los datos por clase
    class_data = df_dialogo[df_dialogo['tipo_num'] == class_label]

    # Preparar los datos de la clase para evaluar
    tokens = tokenizer_TF.batch_encode_plus(
        class_data['palabras'].tolist(),
        truncation=True,
        padding=True,
        return_tensors='pt'
    )

    inputs = tokens['input_ids']
    attention_mask = tokens['attention_mask']
    labels = class_data['tipo_num'].tolist()

    # Pasar los datos de la clase por el modelo
    with torch.no_grad():
        outputs = Modelo_TF(inputs, attention_mask=attention_mask)

    predicted_labels = outputs.logits.argmax(dim=1).tolist()

    # Calcular la precisión para la clase
    accuracy = accuracy_score(labels, predicted_labels)
    print(f"Precisión por clase {df_dialogo[df_dialogo.tipo_num == class_label]['tipo'].unique()[0]}: {accuracy}")

Precisión por clase Sentimiento: 0.9436619718309859
Precisión por clase Saludos: 0.95
Precisión por clase Nombre: 0.9583333333333334
Precisión por clase Identidad: 0.9382716049382716
Precisión por clase ElProfeAlejo: 0.9259259259259259
Precisión por clase Aprendizaje: 0.9302325581395349
Precisión por clase Funcion: 0.8767123287671232
Precisión por clase Usuario: 0.7567567567567568
Precisión por clase Origen: 0.8
Precisión por clase Edad: 0.7741935483870968
Precisión por clase Agradecimiento: 0.9545454545454546
Precisión por clase Continuacion: 0.9032258064516129
Precisión por clase Contacto: 0.3448275862068966
Precisión por clase Despedida: 0.9433962264150944
Precisión por clase Otros: 0.9226006191950464
Precisión por clase Error: 0.875


In [ ]:
# Procesar nueva frase
frase = normalizar('donde vives?')

# Tokenizar la frase de entrada
tokens = tokenizer_TF.encode_plus(
    frase,
    add_special_tokens=True,
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Obtener los input_ids y attention_mask
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

# Realizar la predicción
with torch.no_grad():
    outputs = Modelo_TF(input_ids, attention_mask)

# Obtener las etiquetas predichas
etiquetas_predichas = torch.argmax(outputs.logits, dim=1)

# Decodificar las etiquetas predichas
etiquetas_decodificadas = etiquetas_predichas.tolist()
print("La frase", frase, "se clasifica como: ", df_dialogo[df_dialogo.tipo_num == etiquetas_decodificadas[0]]['tipo'].unique()[0])